In [1]:
import pandas as pd
import numpy as np
import glob
import os

# Carrega a tabela de modelagem mais recente, que já inclui as features de clima
path_base = '../data/processed/modeling_table_final.parquet' 
df_base = pd.read_parquet(path_base)
print(f"Tabela base carregada com {df_base.shape[0]} registros.")

Tabela base carregada com 5693 registros.


In [2]:
# --- Função para carregar múltiplos ficheiros CSV ---
def carregar_dados_anuais(caminho_padrao):
    """Encontra, lê e concatena múltiplos ficheiros CSV anuais."""
    lista_arquivos = glob.glob(caminho_padrao)
    if not lista_arquivos:
        raise FileNotFoundError(f"Nenhum ficheiro encontrado para o padrão: {caminho_padrao}")
        
    print(f"Encontrados {len(lista_arquivos)} ficheiros de queimadas para processar.")
    
    lista_dfs = []
    for arquivo in sorted(lista_arquivos): # Usamos sorted() para manter a ordem cronológica
        print(f"Lendo o ficheiro: {os.path.basename(arquivo)}")
        try:
            df_ano = pd.read_csv(arquivo)
            lista_dfs.append(df_ano)
        except Exception as e:
            print(f"AVISO: Falha ao ler o ficheiro {arquivo}. Erro: {e}")
            
    return pd.concat(lista_dfs, ignore_index=True) if lista_dfs else pd.DataFrame()

# --- Execução do Carregamento ---
try:
    padrao_queimadas = '../data/raw/FOCOS_DE_INCENCIO_GO_*.csv'
    df_queimadas_raw = carregar_dados_anuais(padrao_queimadas)
    
    if not df_queimadas_raw.empty:
        print(f"\n-> Dados de queimadas carregados com sucesso. Total de {len(df_queimadas_raw)} focos encontrados.")
    else:
        print("\nAVISO: Nenhum dado de queimadas foi carregado.")

except Exception as e:
    print(f"\nERRO CRÍTICO ao carregar dados de queimadas: {e}")
    raise

Encontrados 16 ficheiros de queimadas para processar.
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2010.csv
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2011.csv
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2012.csv
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2013.csv
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2014.csv
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2015.csv
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2016.csv
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2017.csv
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2018.csv
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2019.csv
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2020.csv
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2021.csv
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2022.csv
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2023.csv
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2024.csv
Lendo o ficheiro: FOCOS_DE_INCENCIO_GO_2025.csv

-> Dados de queimadas carregados com sucesso. Total de 89383 focos encontrados.


In [3]:
if not df_queimadas_raw.empty:
    df_queimadas = df_queimadas_raw.copy()

    # Converte a coluna de data para o formato datetime
    df_queimadas['timestamp'] = pd.to_datetime(df_queimadas['DataHora'])
    # Extrai apenas a data, sem a hora, para podermos agrupar por dia
    df_queimadas['data'] = df_queimadas['timestamp'].dt.date

    # Agrega os dados por dia, criando nossas novas features
    df_queimadas_diario = df_queimadas.groupby('data').agg(
        focos_incendio_count=('DataHora', 'count'),  # Conta o número de focos no dia
        risco_fogo_medio=('RiscoFogo', 'mean'),        # Média do risco de fogo no dia
        frp_total_diario=('FRP', 'sum'),            # Soma da intensidade (FRP) dos focos do dia
        dias_sem_chuva_max=('DiaSemChuva', 'max')      # Pega o valor máximo de dias sem chuva reportado no dia
    ).reset_index()

    # Converte a coluna 'data' de volta para o formato datetime e a define como índice
    df_queimadas_diario['timestamp'] = pd.to_datetime(df_queimadas_diario['data'])
    df_queimadas_diario.set_index('timestamp', inplace=True)
    df_queimadas_diario = df_queimadas_diario.drop('data', axis=1)

    # Garante que todos os dias do nosso período base estejam presentes, preenchendo com 0
    # os dias em que não houve nenhum foco de incêndio.
    df_queimadas_final = df_queimadas_diario.reindex(df_base.index).fillna(0)

    print("Features diárias de queimadas criadas com sucesso:")
    display(df_queimadas_final.head())
else:
    print("Nenhum dado de queimadas para processar. Criando um DataFrame vazio.")
    df_queimadas_final = pd.DataFrame()

Features diárias de queimadas criadas com sucesso:


,focos_incendio_count,risco_fogo_medio,frp_total_diario,dias_sem_chuva_max
timestamp,,,,
2010-02-10,0.0,0.0,0.0,0.0
2010-02-11,2.0,0.0,18.6,0.0
2010-02-12,0.0,0.0,0.0,0.0
2010-02-13,3.0,0.0,36.2,0.0
2010-02-14,0.0,0.0,0.0,0.0


In [4]:
# Começa com a nossa tabela base
df_final_externas = df_base.copy()

# Adiciona as novas features de queimadas, se existirem
if not df_queimadas_final.empty:
    df_final_externas = df_final_externas.join(df_queimadas_final)
    # Trata possíveis nulos que possam ter surgido na junção
    df_final_externas.fillna(0, inplace=True)
    print("Tabela final enriquecida com dados de queimadas:")
else:
    print("Nenhuma feature de queimadas foi adicionada.")

display(df_final_externas.head())

# Salvar a nova tabela, que será a mais completa de todas
output_path = '../data/processed/modeling_table_externas.parquet'
df_final_externas.to_parquet(output_path)
print(f"\nTABELA FINAL PARA MODELAGEM salva com sucesso em: {output_path}")

Tabela final enriquecida com dados de queimadas:


,deficit_diario_mwh,nivel_risco,programada,verificada,diferenca_verif_prog,geracao_total_diaria_go,geracao_eolielétrica_diaria,geracao_fotovoltaica_diaria,geracao_hidroelétrica_diaria,geracao_nuclear_diaria,...,disponibilidade_total_diaria_go,ghi,temp2m_c,precipitacao_mm,precip_acumulada_14d,precip_acumulada_30d,focos_incendio_count,risco_fogo_medio,frp_total_diario,dias_sem_chuva_max
timestamp,,,,,,,,,,,,,,,,,,,,,
2010-02-10,7.00,baixo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,66008.92373,6.356550,23.4150,5.7350,44.5025,128.0675,0.0,0.0,0.0,0.0
2010-02-11,0.00,baixo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,66008.92373,5.414150,23.1925,4.9250,45.4575,129.4475,2.0,0.0,18.6,0.0
2010-02-12,0.00,baixo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,66008.92373,6.258125,23.6650,0.5475,41.4950,125.4625,0.0,0.0,0.0,0.0
2010-02-13,0.00,baixo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,66008.92373,6.123075,24.4625,0.9875,40.1350,121.9375,3.0,0.0,36.2,0.0
2010-02-14,0.75,baixo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,66008.92373,5.725550,24.4800,1.1650,40.2125,117.8125,0.0,0.0,0.0,0.0



TABELA FINAL PARA MODELAGEM salva com sucesso em: ../data/processed/modeling_table_externas.parquet
